# Script to update the "unplugged" GBR models and to produce extended climate timeseries for input into other GBR models


### Unplugged Source models
* The unplugged Source models are located on Athena here: \\athena\pdreef\Source_Modelling\Climate_extension\Source_models
* Read this document for information on the types of unplugged models: \\athena\pdreef\Source_Modelling\Climate_extension\Source_models\READ_ME_Origin_of_Source_models.docx

### Climate storage relationships
* The climate storage relationships can be found here: \\athena\pdreef\Source_Modelling\Climate_extension\Storage_climate_sources

## Script information
This script uses a "dummy" Source model to generate climate timeseries that can be used to extend the climate on a Source model of your choice. 

You will need to have veneer-py and the Veneer plugin installed to use this script. Refer https://github.com/flowmatters/veneer-py/blob/master/doc/training/0Setup_and_Hello_World.ipynb to for instructions on how to install Veneer.

### Before running this script!!
* Open a generic Source model that has the DS plugin and functionality. Note this model does not need to be the model you are intending to extend. It can be a much smaller model, for example, the Beckers model (\\athena\pdreef\Source_Modelling\Beckers\Beckers_GBR_411_Seasonal.rsproj)
* In the model, go to the 'Tools' menu, select 'Web Server Monitoring'. Choose a port number, check 'Allow Scripts' and start the server connection.

In [1]:
import veneer
import os
import pandas as pd
import numpy as np
import sys

import dsed
from dsed import preprocessors

from datetime import datetime,timedelta

climate Dynamic_SedNet.Parameterisation.Models.ClimateCollationModel
gully Dynamic_SedNet.Parameterisation.Models.GullyParameterisationModel
usle Dynamic_SedNet.Parameterisation.Models.CoverTimeSeries_SpatialPreprocessorModel
gbrusle GBR_DynSed_Extension.Parameterisation.Models.GBRUSLECoverTimeSeriesSpatialPreprocessorModel


### Set the port number below according to the one that is active in your 'Web Server Monitoring' window

In [2]:
PORT=10000
v = veneer.Veneer(port=PORT)

In [3]:
veneer.general.PRINT_SCRIPTS=False

## Comment out the regions you do not wish to process data for

In [4]:
## Comment out the regions below that you do not wish to process data for

REGIONs = ['BU_with_SC1',
           'BU_without_SC1',
           'WT',
           'CY',
           'MW',
           'BM'
           'FI',
          ]

In [5]:
%xmode Verbose

Exception reporting mode: Verbose


## Climate inputs (SILO download)
* Climate data downloaded from SILO (http://siloprod/silo/spatial.html) for the extents specified in: \\athena\pdreef\Source_Modelling\Model_Inputs\ClimateInputs\SILOdumps\extent_rectangle.xls

## Settings
Check and change the file paths below as needed.

#### Unzip the downloaded silo pet and rain files in their respective folders for each region. Call the unzipped pet folder 'pet_grids' and the unzipped rain folder 'rain_grids'.

#### Set the local directory and folder where the SILO downloaded and UNZIPPED rain grids and pet grids are located as 'local_climate_dir' in the cell below.

In [6]:
#Set the start and end date of your required climate TS (typically is the period of your downloaded SILO climate files)
StartDate = '01/01/1960'
EndDate = '31/03/2017'

local_climate_dir = 'D:/Climate_extension/SILOdumps/Jan1960_Mar2018'

os.makedirs(local_climate_dir +'/'+ 'BU', exist_ok=True)
os.makedirs(local_climate_dir +'/'+'WT', exist_ok=True)
os.makedirs(local_climate_dir +'/'+'CY', exist_ok=True)
os.makedirs(local_climate_dir +'/'+'MW', exist_ok=True)
os.makedirs(local_climate_dir +'/'+'FI', exist_ok=True)
os.makedirs(local_climate_dir +'/'+'BM', exist_ok=True)


In [7]:
# Set up the local outputTS directory from which output folders for each region will be created. 
outputTS_dir = 'D:/Climate_extension/outputTS/Jan1960_Mar2018'

# Create new output TS folders for each region
os.makedirs(outputTS_dir +'/'+ 'BU_with_SC1', exist_ok=True)
os.makedirs(outputTS_dir +'/'+ 'BU_without_SC1', exist_ok=True)
os.makedirs(outputTS_dir +'/'+'WT', exist_ok=True)
os.makedirs(outputTS_dir +'/'+'CY', exist_ok=True)
os.makedirs(outputTS_dir +'/'+'MW', exist_ok=True)
os.makedirs(outputTS_dir +'/'+'FI', exist_ok=True)
os.makedirs(outputTS_dir +'/'+'BM', exist_ok=True)

In [8]:
climcoords_path = '//athena/pdreef/Source_Modelling/Model_Inputs/ClimateInputs/CatchModCSVs/'
subCatchmentPolys_path = '//athena/pdreef/Source_Modelling/Model_Inputs/ClimateInputs/CatchModCSVs/catchmentShapes/'

## Script that loops through the regions specified at the start of the notebook to produce the PET and Rain files for each SC

In [9]:
# Specifies the required files and parameters to run the climate preprocessor for each of the regions

for region in REGIONs:
    if region=='BU_with_SC1':
        rainfallInputDirectory = local_climate_dir+'/BU'+'/rain_grids'
        petInputDirectory = local_climate_dir+'/BU'+'/pet_grids'
        subCatchmentPolys = 'BU/Exported ReportingRegions.shp'
        subCatchFieldName = '"Catchmt"'
        outputDIR = os.path.join(outputTS_dir + '/','BU_with_SC1')
        geoCoordFileName = 'BU_RC8_30m_climcoords_IncludingSC1.csv'
        rainScalingFile = '""'
        parameters = {
            'updateModels':False,
            'useInternalSubCatchmentPolys':False,
            'Path_SubCatchPolys':'"' + os.path.join(subCatchmentPolys_path,subCatchmentPolys) +'"',
            'SubCatchPolys_FieldName':subCatchFieldName,
            'OutputDirectory':'"'+ outputDIR +'"',
            'coordsFileName':'"'+ os.path.join(climcoords_path, geoCoordFileName) +'"',
            'rainfallInputDirectory':'"' + rainfallInputDirectory + '"',
            'rainfallDatePattern':'"yyyymmdd"',
            'petInputDirectory':'"' + petInputDirectory + '"',
            'petDatePattern':'"yyyymmdd"',
            'SaveTimeSeries':True,
            'TSonly':False,    
            'useExistingGeoCoords':True,
            'saveCoordsFile':False,    
            'projectionString':'"Australian Albers"',
            'cellSize':30,
            'appendToExisting':False,
            'StartDateString':'"' + StartDate + '"',
            'EndDateString':'"' + EndDate + '"',
            'Path_RainScalingFile':rainScalingFile
        }
        
        preprocessor = preprocessors.run_climate
        UNUSED = []
        defaults = preprocessors.climate_default_params(v)
        {k:v for k,v in defaults.items() if v is None and not k in parameters and not k in UNUSED}
        veneer.general.PRINT_SCRIPTS=True
        result = preprocessors.run_climate(v,**parameters)

        
    elif region=='BU_without_SC1':
        rainfallInputDirectory = local_climate_dir+'/BU'+'/rain_grids'
        petInputDirectory = local_climate_dir+'/BU'+'/pet_grids'
        subCatchmentPolys = 'BU/Exported ReportingRegions.shp' # -- Need the shapefile for BU without SC#1!!
        subCatchFieldName = '"Catchmt"'
        outputDIR = os.path.join(outputTS_dir + '/', 'BU_without_SC1')
        geoCoordFileName = 'BU_RC8_30m_climcoords_NoSC1.csv'
        rainScalingFile = '""'
        parameters = {
            'updateModels':False,
            'useInternalSubCatchmentPolys':False,
            'Path_SubCatchPolys':'"' + os.path.join(subCatchmentPolys_path,subCatchmentPolys) +'"',
            'SubCatchPolys_FieldName':subCatchFieldName,
            'OutputDirectory':'"'+ outputDIR +'"',
            'coordsFileName':'"'+ os.path.join(climcoords_path, geoCoordFileName) +'"',
            'rainfallInputDirectory':'"' + rainfallInputDirectory + '"',
            'rainfallDatePattern':'"yyyymmdd"',
            'petInputDirectory':'"' + petInputDirectory + '"',
            'petDatePattern':'"yyyymmdd"',
            'SaveTimeSeries':True,
            'TSonly':False,    
            'useExistingGeoCoords':True,
            'saveCoordsFile':False,    
            'projectionString':'"Australian Albers"',
            'cellSize':30,
            'appendToExisting':False,
            'StartDateString':'"' + StartDate + '"',
            'EndDateString':'"' + EndDate + '"',
            'Path_RainScalingFile':rainScalingFile
        }
        
        preprocessor = preprocessors.run_climate
        UNUSED = []
        defaults = preprocessors.climate_default_params(v)
        {k:v for k,v in defaults.items() if v is None and not k in parameters and not k in UNUSED}
        veneer.general.PRINT_SCRIPTS=True
        result = preprocessors.run_climate(v,**parameters)
        

    elif region=='WT':
        rainfallInputDirectory = local_climate_dir+'/WT'+'/rain_grids'
        petInputDirectory = local_climate_dir+'/WT'+'/pet_grids'
        subCatchmentPolys = 'WT/Exported ReportingRegions.shp'
        subCatchFieldName = '"Catchmt"'
        outputDIR = os.path.join(outputTS_dir + '/','WT')
        geoCoordFileName = 'WT_RC8_30m_climcoords.csv'
        rainScalingFile = '"' + os.path.join(climcoords_path,'wt_rain_scaling_factors.csv') + '"' 
        parameters = {
            'updateModels':False,
            'useInternalSubCatchmentPolys':False,
            'Path_SubCatchPolys':'"' + os.path.join(subCatchmentPolys_path,subCatchmentPolys) +'"',
            'SubCatchPolys_FieldName':subCatchFieldName,
            'OutputDirectory':'"'+ outputDIR +'"',
            'coordsFileName':'"'+ os.path.join(climcoords_path, geoCoordFileName) +'"',
            'rainfallInputDirectory':'"' + rainfallInputDirectory + '"',
            'rainfallDatePattern':'"yyyymmdd"',
            'petInputDirectory':'"' + petInputDirectory + '"',
            'petDatePattern':'"yyyymmdd"',
            'SaveTimeSeries':True,
            'TSonly':False,    
            'useExistingGeoCoords':True,
            'saveCoordsFile':False,    
            'projectionString':'"Australian Albers"',
            'cellSize':30,
            'appendToExisting':False,
            'StartDateString':'"' + StartDate + '"',
            'EndDateString':'"' + EndDate + '"',
            'Path_RainScalingFile':rainScalingFile
        }
        
        preprocessor = preprocessors.run_climate
        UNUSED = []
        defaults = preprocessors.climate_default_params(v)
        {k:v for k,v in defaults.items() if v is None and not k in parameters and not k in UNUSED}
        veneer.general.PRINT_SCRIPTS=True
        result = preprocessors.run_climate(v,**parameters)
        
        
    elif region=='CY':
        rainfallInputDirectory = local_climate_dir+'/CY'+'/rain_grids'
        petInputDirectory = local_climate_dir+'/CY'+'/pet_grids'
        subCatchmentPolys = 'CY/Exported ReportingRegions.shp'
        subCatchFieldName = '"Catchmt"'
        outputDIR = os.path.join(outputTS_dir + '/','CY')
        geoCoordFileName = 'CY_RC8_30m_climcoords.csv'
        rainScalingFile = '""'
        parameters = {
            'updateModels':False,
            'useInternalSubCatchmentPolys':False,
            'Path_SubCatchPolys':'"' + os.path.join(subCatchmentPolys_path,subCatchmentPolys) +'"',
            'SubCatchPolys_FieldName':subCatchFieldName,
            'OutputDirectory':'"'+ outputDIR +'"',
            'coordsFileName':'"'+ os.path.join(climcoords_path, geoCoordFileName) +'"',
            'rainfallInputDirectory':'"' + rainfallInputDirectory + '"',
            'rainfallDatePattern':'"yyyymmdd"',
            'petInputDirectory':'"' + petInputDirectory + '"',
            'petDatePattern':'"yyyymmdd"',
            'SaveTimeSeries':True,
            'TSonly':False,    
            'useExistingGeoCoords':True,
            'saveCoordsFile':False,    
            'projectionString':'"Australian Albers"',
            'cellSize':30,
            'appendToExisting':False,
            'StartDateString':'"' + StartDate + '"',
            'EndDateString':'"' + EndDate + '"',
            'Path_RainScalingFile':rainScalingFile
        }
        
        preprocessor = preprocessors.run_climate
        UNUSED = []
        defaults = preprocessors.climate_default_params(v)
        {k:v for k,v in defaults.items() if v is None and not k in parameters and not k in UNUSED}
        veneer.general.PRINT_SCRIPTS=True
        result = preprocessors.run_climate(v,**parameters)

        
    elif region=='MW':
        rainfallInputDirectory = local_climate_dir+'/MW'+'/rain_grids'
        petInputDirectory = local_climate_dir+'/MW'+'/pet_grids'
        subCatchmentPolys = 'MW/Exported ReportingRegions.shp'
        subCatchFieldName = '"Catchmt"'
        outputDIR = os.path.join(outputTS_dir + '/','MW')
        geoCoordFileName = 'MW_RC8_30m_climcoords.csv'
        rainScalingFile = '""'
        parameters = {
            'updateModels':False,
            'useInternalSubCatchmentPolys':False,
            'Path_SubCatchPolys':'"' + os.path.join(subCatchmentPolys_path,subCatchmentPolys) +'"',
            'SubCatchPolys_FieldName':subCatchFieldName,
            'OutputDirectory':'"'+ outputDIR +'"',
            'coordsFileName':'"'+ os.path.join(climcoords_path, geoCoordFileName) +'"',
            'rainfallInputDirectory':'"' + rainfallInputDirectory + '"',
            'rainfallDatePattern':'"yyyymmdd"',
            'petInputDirectory':'"' + petInputDirectory + '"',
            'petDatePattern':'"yyyymmdd"',
            'SaveTimeSeries':True,
            'TSonly':False,    
            'useExistingGeoCoords':True,
            'saveCoordsFile':False,    
            'projectionString':'"Australian Albers"',
            'cellSize':30,
            'appendToExisting':False,
            'StartDateString':'"' + StartDate + '"',
            'EndDateString':'"' + EndDate + '"',
            'Path_RainScalingFile':rainScalingFile
        }
        
        preprocessor = preprocessors.run_climate
        UNUSED = []
        defaults = preprocessors.climate_default_params(v)
        {k:v for k,v in defaults.items() if v is None and not k in parameters and not k in UNUSED}
        veneer.general.PRINT_SCRIPTS=True
        result = preprocessors.run_climate(v,**parameters)
        

    elif region=='FI':
        rainfallInputDirectory = local_climate_dir+'/FI'+'/rain_grids'
        petInputDirectory = local_climate_dir+'/FI'+'/pet_grids'
        subCatchmentPolys = 'FI/Exported ReportingRegions.shp'
        subCatchFieldName = '"Catchmt"'
        outputDIR = os.path.join(outputTS_dir + '/','FI')
        geoCoordFileName = 'FI_RC8_30m_climcoords.csv'
        rainScalingFile = '""'
        parameters = {
            'updateModels':False,
            'useInternalSubCatchmentPolys':False,
            'Path_SubCatchPolys':'"' + os.path.join(subCatchmentPolys_path,subCatchmentPolys) +'"',
            'SubCatchPolys_FieldName':subCatchFieldName,
            'OutputDirectory':'"'+ outputDIR +'"',
            'coordsFileName':'"'+ os.path.join(climcoords_path, geoCoordFileName) +'"',
            'rainfallInputDirectory':'"' + rainfallInputDirectory + '"',
            'rainfallDatePattern':'"yyyymmdd"',
            'petInputDirectory':'"' + petInputDirectory + '"',
            'petDatePattern':'"yyyymmdd"',
            'SaveTimeSeries':True,
            'TSonly':False,    
            'useExistingGeoCoords':True,
            'saveCoordsFile':False,    
            'projectionString':'"Australian Albers"',
            'cellSize':30,
            'appendToExisting':False,
            'StartDateString':'"' + StartDate + '"',
            'EndDateString':'"' + EndDate + '"',
            'Path_RainScalingFile':rainScalingFile
        }
        
        preprocessor = preprocessors.run_climate
        UNUSED = []
        defaults = preprocessors.climate_default_params(v)
        {k:v for k,v in defaults.items() if v is None and not k in parameters and not k in UNUSED}
        veneer.general.PRINT_SCRIPTS=True
        result = preprocessors.run_climate(v,**parameters)
        
        
    elif region=='BM':
        rainfallInputDirectory = local_climate_dir+'/BM'+'/rain_grids'
        petInputDirectory = local_climate_dir+'/BM'+'/pet_grids'
        subCatchmentPolys = 'BM/Exported ReportingRegions.shp'
        subCatchFieldName = '"Catchmt"'
        outputDIR = os.path.join(outputTS_dir + '/','BM')
        geoCoordFileName = 'BM_RC8_30m_climcoords.csv'
        rainScalingFile = '""'
        rainScalingFile = '""'
        parameters = {
            'updateModels':False,
            'useInternalSubCatchmentPolys':False,
            'Path_SubCatchPolys':'"' + os.path.join(subCatchmentPolys_path,subCatchmentPolys) +'"',
            'SubCatchPolys_FieldName':subCatchFieldName,
            'OutputDirectory':'"'+ outputDIR +'"',
            'coordsFileName':'"'+ os.path.join(climcoords_path, geoCoordFileName) +'"',
            'rainfallInputDirectory':'"' + rainfallInputDirectory + '"',
            'rainfallDatePattern':'"yyyymmdd"',
            'petInputDirectory':'"' + petInputDirectory + '"',
            'petDatePattern':'"yyyymmdd"',
            'SaveTimeSeries':True,
            'TSonly':False,    
            'useExistingGeoCoords':True,
            'saveCoordsFile':False,    
            'projectionString':'"Australian Albers"',
            'cellSize':30,
            'appendToExisting':False,
            'StartDateString':'"' + StartDate + '"',
            'EndDateString':'"' + EndDate + '"',
            'Path_RainScalingFile':rainScalingFile
        }
        
        preprocessor = preprocessors.run_climate
        UNUSED = []
        defaults = preprocessors.climate_default_params(v)
        {k:v for k,v in defaults.items() if v is None and not k in parameters and not k in UNUSED}
        veneer.general.PRINT_SCRIPTS=True
        result = preprocessors.run_climate(v,**parameters)
        

# Generated Script
import Dynamic_SedNet.Parameterisation.Models.ClimateCollationModel
import clr
clr.AddReference('System.Core')
import System
import FlowMatters.Source.Veneer.RemoteScripting.ScriptHelpers as H
clr.ImportExtensions(System.Linq)
from TIME.Core.Metadata import OutputAttribute
result = []
tmp = Dynamic_SedNet.Parameterisation.Models.ClimateCollationModel()
typeObject = tmp.GetType()
for member in dir(tmp):
  try:
    if typeObject.GetMember(member)[0].IsDefined(OutputAttribute,True):
      result.append(member)
  except: pass

import Dynamic_SedNet.Parameterisation.Models.ClimateCollationModel as ClimateCollationModel
import Dynamic_SedNet.Tools.ToolsModel as ToolsModel

p = ClimateCollationModel()
p.Scenario = scenario
p.CatPolys = scenario.GeographicData.IntersectedSCFUPolygons[1]
p.FuPolys = scenario.GeographicData.IntersectedSCFUPolygons[1]
p.updateModels = False
p.useInternalSubCatchmentPolys = False
p.Path_SubCatchPolys = "//athena/pdreef/Source_Modelling/Model_Inp